## Simple CNN

In this assignment, you need to build a simple CNN to classify images in CIFAR-10 dataset.

![cifar10](https://pytorch.org/tutorials/_images/cifar10.png)

The framework is given below. What you need to do is to build your net. Then tune the hyperparameters, train the NN, and achieve **60%+** accuracy. (about 1h training on CPU is enough)

You can also refer to the [PyTorch tutorial](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np
from tqdm import tqdm

print(torch.__version__)

1.4.0+cpu


Some hyperparameters. They are maybe not the best, you need to tune, especially the learning rate.

In [2]:
LEARNING_RATE = 1e-3
NUM_EPOCHS = 15
BATCH_SIZE = 32 # Mini-batch size
DEVICE = torch.device('cpu')
CONFIG = {"lr": LEARNING_RATE,
          "num_epochs": NUM_EPOCHS,
          "batch_size": BATCH_SIZE,
          "network": "LeNet feat. Adam"}

**TODO**: Define your NN here.

Some reference networks:
* LeNet
* AlexNet
* VGG
* GoogleLeNet (maybe to large for your PC to train)
* ResNet (too large)

Ref: [Overview of CNN Development - Zhihu](https://zhuanlan.zhihu.com/p/66215918)

Do not directly copy others' code. Check these networks' structure and write code yourself.

In [3]:
class Net(nn.Module):
    """
    TODO: Implementation of a simple Convolutional neural network.
    HINT: You can refer to several famous CNNs, like LeNet5, VGG16, ResNet
        Actually you cannot directly copy the code in the demo,
        since you need to recalculate the shape of the tensor.
    """
    """YOUR CODE HERE"""
    def __init__(self):
        super(Net, self).__init__()
        self.features = nn.Sequential(  #3*32*32
            nn.Conv2d(3, 6, 5),         #6*28*28
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),         #6*14*14
            nn.Conv2d(6, 16, 5),        #16*10*10
            nn.MaxPool2d(2, 2),         #16*5*5
            nn.ReLU(inplace=True)
        )
        
        self.FC = nn.Sequential(  #3*32*32
            nn.Linear(16 * 5 * 5, 120),
            nn.ReLU(inplace=True),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Linear(84, 10)
        )
        
        #self.C5 = nn.Linear(16*5*5, 120)
        #self.F6 = nn.Linear(120, 84),
        #self.Output = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(-1, 16*5*5)
        #x = nn.ReLU(self.C5(x))
        #x = nn.ReLU(self.F6(x))
        #x = self.Output(x)
        x = self.FC(x)
        return x
    """END OF YOUR CODE"""

model = Net().to(device=DEVICE)
print(model)

Net(
  (features): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): ReLU(inplace=True)
  )
  (FC): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


Load CIFAR-10 dataset. Downloading will be started automatically.

In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], # magic numbers
                         std=[0.229, 0.224, 0.225]) # 3 channels
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


Define the loss function.

In [5]:
criterion = F.cross_entropy

Define the optimizer. You can **choose other** optimizers as you like.

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

Evaluation process.

In [7]:
def evaluate(model_eval, loader_eval, criterion_eval):
    model_eval.eval()
    loss_eval = 0
    correct = 0.
    pbar = tqdm(total = len(loader_eval), desc='Evaluation', ncols=100)
    with torch.no_grad():
        for data, target in loader_eval:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            loss_eval += criterion_eval(output, target).item()

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            pbar.update(1)
    pbar.close()

    loss_eval = loss_eval / loader_eval.dataset.__len__()
    accuracy = correct / loader_eval.dataset.__len__()
    response = {'loss': loss_eval, 'acc': accuracy}
    return response

The main training function defined below. Be careful of [overfitting](https://en.wikipedia.org/wiki/Overfitting)!

In [8]:
train_acc = np.zeros(NUM_EPOCHS)
eval_acc = np.zeros(NUM_EPOCHS)
train_loss = np.zeros(NUM_EPOCHS)
eval_loss = np.zeros(NUM_EPOCHS)

model.train()
for epoch_idx in range(NUM_EPOCHS):
    pbar = tqdm(total = len(train_dataloader), desc='Train - Epoch {}'.format(epoch_idx), ncols=100)
    for batch_idx, (data, target) in enumerate(train_dataloader):
        data, target = data.to(DEVICE), target.to(DEVICE)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        pbar.update(1)
    pbar.close()

    train_resp = evaluate(model, train_dataloader, criterion)
    eval_resp = evaluate(model, test_dataloader, criterion)

    print ('-*-*-*-*-*- Epoch {} -*-*-*-*-*-'.format(epoch_idx))
    print ('Train Loss: {:.6f}\t'.format(train_resp['loss']))
    print ('Train Acc: {:.6f}\t'.format(train_resp['acc']))
    print ('Eval Loss: {:.6f}\t'.format(eval_resp['loss']))
    print ('Eval Acc: {:.6f}\t'.format(eval_resp['acc']))
    print ('\n')

    train_acc[epoch_idx] = train_resp['acc']
    eval_acc[epoch_idx] = eval_resp['acc']
    train_loss[epoch_idx] = train_resp['loss']
    eval_loss[epoch_idx] = eval_resp['loss']

    # save model and training data
    torch.save(model, 'simple-cnn.pth'.format(CONFIG["network"]))
    np.savez('simple-cnn',config=CONFIG,train_acc=train_acc, eval_acc=eval_acc,train_loss=train_loss, eval_loss=eval_loss)

Evaluation: 100%|█████████████████████████████████████████████████| 313/313 [00:10<00:00, 30.89it/s]


-*-*-*-*-*- Epoch 0 -*-*-*-*-*-
Train Loss: 0.042686	
Train Acc: 0.503480	
Eval Loss: 0.043640	
Eval Acc: 0.496200	




D:\Downloads\Anaconda3\lib\site-packages\torch\serialization.py:360: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
Train - Epoch 2:   0%|                                             | 2/1563 [00:00<01:27, 17.89it/s]

-*-*-*-*-*- Epoch 1 -*-*-*-*-*-
Train Loss: 0.035422	
Train Acc: 0.593960	
Eval Loss: 0.037461	
Eval Acc: 0.571100	




Train - Epoch 3:   0%|                                             | 2/1563 [00:00<01:20, 19.44it/s]

...

-*-*-*-*-*- Epoch 5 -*-*-*-*-*-
Train Loss: 0.027405	
Train Acc: 0.690860	
Eval Loss: 0.033991	
Eval Acc: 0.621800	




Train - Epoch 7:   0%|                                             | 2/1563 [00:00<01:18, 19.81it/s]

-*-*-*-*-*- Epoch 6 -*-*-*-*-*-
Train Loss: 0.026853	
Train Acc: 0.695220	
Eval Loss: 0.034305	
Eval Acc: 0.621900	




Train - Epoch 7:  52%|██████████████████████▎                    | 812/1563 [00:40<00:33, 22.18it/s]

KeyboardInterrupt: 

Visualize the training process

In [ ]:
import matplotlib.pyplot as plt

train_acc = np.zeros(NUM_EPOCHS)
eval_acc = np.zeros(NUM_EPOCHS)
train_loss = np.zeros(NUM_EPOCHS)
eval_loss = np.zeros(NUM_EPOCHS)

def plot_acc(train_acc,eval_acc):
    plt.plot(train_acc,label="Train")
    plt.plot(eval_acc,label="Test")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend()
    plt.show()

def plot_loss(train_loss,eval_loss):
    plt.plot(train_loss,label="Train")
    plt.plot(eval_loss,label="Test")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()
    
plot_acc(train_acc,eval_acc)
plot_loss(train_loss,eval_loss)